In [1]:
!pip install pywhatkit

  Using cached pywhatkit-4.7-py3-none-any.whl (10 kB)
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Using cached PyAutoGUI-0.9.52.tar.gz (55 kB)
  Using cached PyMsgBox-1.0.9.tar.gz (18 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 292 kB 672 kB/s eta 0:00:01
     |████████████████████████████████| 53 kB 988 kB/s eta 0:00:01
     |████████████████████████████████| 143 kB 336 kB/s eta 0:00:01
     |████████████████████████████████| 273 kB 364 kB/s eta 0:00:01
  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached soupsieve-2.2.1-py3-none-any.whl (33 kB)
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.52-py3-none-any.whl size=35193 sha256=a083c0658baf08675f756ed91c7f1e7a4e49018e29a44b7c28760795dc8c3830
  Stored i

In [1]:
from imutils.video import VideoStream
import cv2
import imutils
import os
import time
import numpy as np
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
import PIL
import numpy as np
import pafy
from ffpyplayer.player import MediaPlayer
import pywhatkit


In [2]:
detectorPaths = {
	"face": "haarcascade_frontalface_default.xml",
	"eyes": "haarcascade_eye.xml",
	"smile": "haarcascade_smile.xml",
}

detector = {}

for (name,path) in detectorPaths.items():
    path = os.path.join("./haarcascades/",path)
    detector[name] = cv2.CascadeClassifier(path)

print(detector)


{'face': <CascadeClassifier 0x7fb301430bb0>, 'eyes': <CascadeClassifier 0x7fb2f9c16cf0>, 'smile': <CascadeClassifier 0x7fb30218ed10>}


In [3]:
#start video stream
model = VGGFace(model= 'resnet50')
# detector = MTCNN()
target_size = (224,224) # output image size
border_rel = 0 # increase or decrease zoom on image


In [4]:

vs = VideoStream(src=0).start()
time.sleep(2.0)




In [5]:
def play_video(img,name):
    
    cv2.imshow("img",img)
    
    url = pywhatkit.playonyt(name) or "https://www.youtube.com/watch?v=vMLk_T0PPbk"
    videoPafy = pafy.new(url)
    best = videoPafy.getbest()

    temp_video=cv2.VideoCapture(best.url)
    player = MediaPlayer(best.url)
    player.set_pause(True)
    
    cap = cv2.VideoCapture(0)
#     img_target = cv2.imread("test2.jpg")
    img_target = img
#     temp_video = cv2.VideoCapture("test.mov")


    detection = False
    frame_counter = 0

    success, imgVideo = temp_video.read()
    hT,wT,cT = img_target.shape
    imgVideo = cv2.resize(imgVideo,(wT,hT))



    orb = cv2.ORB_create(nfeatures=1000)
    kp1, des1 = orb.detectAndCompute(img_target,None)


    while True:
        success, img_web_cam = cap.read()
        img_aug = img_web_cam.copy()
        
        kp2, des2 = orb.detectAndCompute(img_web_cam,None)


    #     img_web_cam = cv2.drawKeypoints(img_web_cam,kp2,None) 

        if detection == False:
            temp_video.set(cv2.CAP_PROP_POS_FRAMES,0)
            frame_counter = 0
            player.set_pause(True)
        else: 
            if frame_counter == temp_video.get(cv2.CAP_PROP_FRAME_COUNT):
                temp_video.set(cv2.CAP_PROP_POS_FRAMES,0)
                frame_counter = 0
            player.set_pause(False)
            success, imgVideo = temp_video.read()
            imgVideo = cv2.resize(imgVideo,(wT,hT))
#             audio_frame, val = player.get_frame()


        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1,des2,k=2)
        good = []



        for m,n in matches:
            if m.distance < 0.75 * n.distance:
                good.append(m)


        if len(good) > 20:
            detection = True

            srcPts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
            dstPts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1,1,2)

            matrix, mask = cv2.findHomography(srcPts,dstPts,cv2.RANSAC,5)


            pts = np.float32([[0,0],[0,hT],[wT,hT],[wT,0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,matrix)

            img2 = cv2.polylines(img_web_cam,[np.int32(dst)],True,(225,0,225),3)

            imgWrap = cv2.warpPerspective(imgVideo,matrix,(img_web_cam.shape[1],img_web_cam.shape[0]))

            mask_new = np.zeros((img_web_cam.shape[0],img_web_cam.shape[1]),np.uint8)

            cv2.fillPoly(mask_new,[np.int32(dst)],(225,225,255))
            maskInv = cv2.bitwise_not(mask_new)
            img_aug = cv2.bitwise_and(img_aug,img_aug,mask=maskInv)
            img_aug = cv2.bitwise_or(imgWrap,img_aug)

    #         cv2.imshow("imgWrap",img_aug)

            cv2.fillConvexPoly(mask_new, dst.astype("int32"), (255, 255, 255),cv2.LINE_AA)


            rect = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
            mask = cv2.dilate(mask_new, rect, iterations=2)


            maskScaled = mask_new.copy() / 255.0
            maskScaled = np.dstack([maskScaled] * 3)


            warpedMultiplied = cv2.multiply(imgWrap.astype("float"), maskScaled)
            imageMultiplied = cv2.multiply(img_aug.astype(float), 1.0 - maskScaled)
            output = cv2.add(warpedMultiplied, imageMultiplied)
            output = output.astype("uint8")

            cv2.imshow("output",output)

        img_features = cv2.drawMatches(img_target,kp1,img_web_cam,kp2,good,None,flags=2)



    #     cv2.imshow("feature",img_features)
    #     cv2.imshow("target_image", img_target)

        cv2.imshow("webcam", img_web_cam)
    #     cv2.imshow("video", imgVideo)
        frame_counter += 1

        key = cv2.waitKey(1) & 0xFF
        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    cv2.destroyAllWindows()
    cap.release()
    return


In [6]:
while True:
    
    frame = vs.read()
    frame = imutils.resize(frame,width=500)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    
    face_rect = detector["face"].detectMultiScale(gray,scaleFactor=1.05, minNeighbors=5, minSize=(30, 30),
                                                  flags=cv2.CASCADE_SCALE_IMAGE)
    for (fX, fY, fW, fH) in face_rect:
        
        cv2.rectangle(gray,(fX, fY), (fX + fW, fY + fH),
                      (0, 255, 0), 2)
        
        x1, y1, width, height = fX, fY, fW, fH
        dw = round(width * border_rel)
        dh = round(height * border_rel)

        x2, y2 = x1 + width + dw, y1 + height + dh
#         face = frame[y1:y2, x1:x2]
        
#         faces = frame[fY : fY + fH , fX:fX + fW]
#         cv2.imshow("faces",faces)
    
    
        
        face = frame[y1:y2, x1:x2]
        face = PIL.Image.fromarray(face)
        face = face.resize((224, 224))
        face = np.asarray(face)

 
        face_pp = face.astype('float32')
        face_pp = np.expand_dims(face_pp, axis = 0)
        face_pp = preprocess_input(face_pp, version = 2)
    
        prediction = model.predict(face_pp)
        results = decode_predictions(prediction)
        
        # Display results
#         cv2.imshow("face",face)
        
        if(results[0][0][1] * 100 > 90):
            
#             ans = input()
            name = results[0][0][0]
            play_video(face,name)
            
            for result in results[0]:
                print ('%s: %.3f%%' % (result[0], result[1]*100))
    
    cv2.imshow("Frame", gray)
    
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()

b' Nina_Watson': 94.739%
b' Marieta_Severo': 0.430%
b' Agnieszka_Grochowska': 0.383%
b' Blanca_Marsillach': 0.194%
b' Ir\xc3\xa1n_Castillo': 0.142%
